In [2]:
import torch
import numpy as np
from utils import get_freest_gpu
import torch.nn.functional as F

In [3]:
from predict_transformer_nonlinear import NonLinearNet

In [4]:
freest_gpu = get_freest_gpu()
freest_gpu

1

In [9]:
torch.cuda.set_device(f'cuda:{freest_gpu}')

In [10]:
net = NonLinearNet(3, 3, 64, 64).cuda()

In [49]:
net.load_state_dict(torch.load('nonlinear_model.parameters'))

<All keys matched successfully>

In [50]:
traj_encoder = net.trajectory_encoder
state_encoder = net.state_encoder

In [51]:
states = torch.Tensor(np.load('states_46000.npy')).to(torch.int64)
states = F.one_hot(states, num_classes=4).view(-1, 150, 100).to(torch.float)

In [14]:
# states_unsqueezed = states.unsqueeze(0)

In [66]:
traj = states[30000]
traj = traj.unsqueeze(0).cuda()

In [37]:
reward_est = net(traj, traj[0])
reward_est

tensor([-0.5987, -0.3665, -0.2859, -0.1955,  0.0129, -0.1713,  0.0129,  0.0129,
        -0.1713,  0.0129,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,
         0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,
         0.0129, -0.1713, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129,
        -0.1713,  0.0129,  0.0129,  0.0129, -0.0649,  0.0129, -0.1713, -0.1713,
         0.0129, -0.1713,  0.0129,  0.0129, -0.0649,  0.0129, -0.1713,  0.0129,
        -0.1713, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,
         0.0129, -0.1713, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129,
        -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129,
        -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129,  0.0129, -0.1713,
         0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,
         0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,  0.0129, -0.1713,
         0.0129, -0.1713,  0.0129, -0.17

In [71]:
state_rep = state_encoder(traj[0][0])
state_rep

tensor([2.9061, 3.6869, 3.2795], device='cuda:1', grad_fn=<AddBackward0>)

In [68]:
traj[0][0].view(5, 5, 4)

tensor([[[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 1., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.]]], device='cuda:1')

In [60]:
traj_rep = traj_encoder(traj)
traj_rep

tensor([[1.0000, 0.9999, 1.0000]], device='cuda:1', grad_fn=<SigmoidBackward0>)